In [1]:
import pandas as pd
import glob
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, BatchNormalization, Dropout
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from imblearn import under_sampling, over_sampling, combine
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OneHotEncoder
from collections import Counter

In [2]:
#reading data

path = '/Users/ahmetokanarik/Desktop/MScThesis/Dataset/UNSQ-NB15'
all_files = glob.glob(path + "/*.csv")
li = []

for filename in all_files:
    df = pd.read_csv(filename,header=None,sep=';',low_memory=False)
    li.append(df)


data = pd.concat(li, axis=0, ignore_index=True)
data.columns = ["srcip","sport","dstip","dsport","proto","state","dur","sbytes","dbytes","sttl","dttl","sloss","dloss","service","Sload","Dload","Spkts","Dpkts","swin","dwin","stcpb","dtcpb","smeansz","dmeansz","trans_depth","res_bdy_len","Sjit","Djit","Stime","Ltime","Sintpkt","Dintpkt","tcprtt","synack","ackdat","is_sm_ips_ports","ct_state_ttl","ct_flw_http_mthd","is_ftp_login","ct_ftp_cmd","ct_srv_src","ct_srv_dst","ct_dst_ltm","ct_src_ ltm","ct_src_dport_ltm","ct_dst_sport_ltm","ct_dst_src_ltm","attack_cat","Label"]


In [3]:
#one-hot encoding
dummies = pd.get_dummies(data[['service','proto','state']])
data.drop(['proto','service','state'],axis=1,inplace=True)
data = pd.concat([data, dummies], axis=1)
data = data[["dtcpb","stcpb","service_-","Dload","dmeansz","service_dns","smeansz","Sload","trans_depth","sttl",
            "service_ftp-data","ct_ftp_cmd","attack_cat"]]

data['attack_cat'] = data['attack_cat'].fillna('Normal')

In [4]:
replace_dict = {np.nan: 0, ' ': 0}
for cols in ['ct_ftp_cmd']:
    data[cols] = data[cols].replace(replace_dict)
    
replace_dict = {np.nan: 0, '0': 0}
for cols in ['ct_ftp_cmd']:
    data[cols] = data[cols].replace(replace_dict)
        

In [5]:
x = data.drop('attack_cat',axis=1).values.astype('float32')
y = data.attack_cat.values

In [6]:
#Target Variable Label
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

In [7]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
x = scaler.fit_transform(x)

In [8]:
x.shape

(2540047, 12)

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x,y,test_size = 0.3,random_state=40,stratify=y)


In [10]:
print(sorted(Counter(y_train).items()))

[(0, 1874), (1, 1630), (2, 11447), (3, 31167), (4, 16972), (5, 150837), (6, 1553134), (7, 9791), (8, 1058), (9, 122)]


In [11]:
a = 177803 #y-traine göre belirlenmeli.177803
smo = SMOTE(sampling_strategy={0:a,1:a,2:a,3:a,4:a,5:a,7:a,8:a,9:a},random_state=42) 
X_train, y_train = smo.fit_resample(X_train, y_train)   
print(sorted(Counter(y_train).items()))

[(0, 177803), (1, 177803), (2, 177803), (3, 177803), (4, 177803), (5, 177803), (6, 1553134), (7, 177803), (8, 177803), (9, 177803)]


In [12]:
from imblearn.under_sampling import NearMiss

# define the undersampling method
undersample = NearMiss(version=1, n_neighbors=3, sampling_strategy={6:a})
X_train, y_train = undersample.fit_resample(X_train, y_train)


In [13]:
print(sorted(Counter(y_train).items()))

[(0, 177803), (1, 177803), (2, 177803), (3, 177803), (4, 177803), (5, 177803), (6, 177803), (7, 177803), (8, 177803), (9, 177803)]


In [14]:
import pandas as pd
from pandas.api.types import is_numeric_dtype
from sklearn.model_selection import train_test_split
import catboost as cb
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from keras.utils import np_utils
from catboost import Pool, CatBoostClassifier

In [15]:
train_dataset = Pool(data = X_train,
                    label=y_train)

eval_dataset = Pool(data=X_test,
                   label=y_test)

In [16]:
"""model = CatBoostClassifier(loss_function='MultiClass',
                           objective='MultiClass',
                          eval_metric='Accuracy',
                          depth=10,
                          l2_leaf_reg=1,
                          iterations=150,
                          learning_rate=0.1)"""

"model = CatBoostClassifier(loss_function='MultiClass',\n                           objective='MultiClass',\n                          eval_metric='Accuracy',\n                          depth=10,\n                          l2_leaf_reg=1,\n                          iterations=150,\n                          learning_rate=0.1)"

In [17]:
"""import time

time_start = time.time()

model.fit(train_dataset)

time_end = time.time()
train_time = time_end - time_start
print("Train time:",train_time)
"""

'import time\n\ntime_start = time.time()\n\nmodel.fit(train_dataset)\n\ntime_end = time.time()\ntrain_time = time_end - time_start\nprint("Train time:",train_time)\n'

In [18]:
"""import time
time_start = time.time()

pred = model.predict(X_test)

time_end = time.time()
test_time = time_end - time_start
print("test_time:",test_time)"""

'import time\ntime_start = time.time()\n\npred = model.predict(X_test)\n\ntime_end = time.time()\ntest_time = time_end - time_start\nprint("test_time:",test_time)'

In [19]:
"""from sklearn import metrics
from sklearn.metrics import classification_report

target_names = ['Analysis','Backdoor','DoS','Exploits','Fuzzers','Generic','Normal','Reconnaissance','Shellcode','Worms']
acc = metrics.accuracy_score(y_test,pred) * 100
f1 = metrics.f1_score(y_test, pred,average='weighted')* 100
pre = metrics.precision_score(y_test, pred, labels=None, pos_label=1, average='weighted') * 100 #DR
recall = metrics.recall_score(y_test, pred, labels=None, pos_label=1, average='weighted', sample_weight=None) * 100

print(classification_report(y_test,pred,target_names=target_names))
print("acc:",acc)
print("pre:",pre)
print("DR=recall:",recall)
print("f1:",f1)
"""

'from sklearn import metrics\nfrom sklearn.metrics import classification_report\n\ntarget_names = [\'Analysis\',\'Backdoor\',\'DoS\',\'Exploits\',\'Fuzzers\',\'Generic\',\'Normal\',\'Reconnaissance\',\'Shellcode\',\'Worms\']\nacc = metrics.accuracy_score(y_test,pred) * 100\nf1 = metrics.f1_score(y_test, pred,average=\'weighted\')* 100\npre = metrics.precision_score(y_test, pred, labels=None, pos_label=1, average=\'weighted\') * 100 #DR\nrecall = metrics.recall_score(y_test, pred, labels=None, pos_label=1, average=\'weighted\', sample_weight=None) * 100\n\nprint(classification_report(y_test,pred,target_names=target_names))\nprint("acc:",acc)\nprint("pre:",pre)\nprint("DR=recall:",recall)\nprint("f1:",f1)\n'

In [20]:
"""import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope, as_apply

space={ 'depth': scope.int(hp.uniform('depth', 1, 11)),
        'l2_leaf_reg': scope.int(hp.uniform('l2_leaf_reg', 1, 11)),
        'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(0.5)) - 0.0001,
        'min_child_samples' : scope.int(hp.loguniform("min_child_samples", np.log(1), np.log(100))),#
        'colsample_bylevel' : hp.uniform('colsample_bylevel', 0.5,1),
      }

# Classifier:
def hyperparameter_tuning(space):
    model = CatBoostClassifier(objective='MultiClass',
                              depth = space['depth'],
                              l2_leaf_reg=space['l2_leaf_reg'],
                              learning_rate=space['learning_rate'],
                              min_child_samples=space['min_child_samples'],
                              colsample_bylevel=space['colsample_bylevel'])
    
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    model.fit(X_train, y_train,verbose=True)

    pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, pred>0.5)
    print ("SCORE:", accuracy)
    #change the metric if you like
    return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}"""

'import hyperopt\nfrom hyperopt import fmin, tpe, hp, STATUS_OK, Trials\nfrom hyperopt.pyll import scope, as_apply\n\nspace={ \'depth\': scope.int(hp.uniform(\'depth\', 1, 11)),\n        \'l2_leaf_reg\': scope.int(hp.uniform(\'l2_leaf_reg\', 1, 11)),\n        \'learning_rate\': hp.loguniform(\'learning_rate\', np.log(0.0001), np.log(0.5)) - 0.0001,\n        \'min_child_samples\' : scope.int(hp.loguniform("min_child_samples", np.log(1), np.log(100))),#\n        \'colsample_bylevel\' : hp.uniform(\'colsample_bylevel\', 0.5,1),\n      }\n\n# Classifier:\ndef hyperparameter_tuning(space):\n    model = CatBoostClassifier(objective=\'MultiClass\',\n                              depth = space[\'depth\'],\n                              l2_leaf_reg=space[\'l2_leaf_reg\'],\n                              learning_rate=space[\'learning_rate\'],\n                              min_child_samples=space[\'min_child_samples\'],\n                              colsample_bylevel=space[\'colsample_bylevel\'

In [21]:
"""from sklearn.metrics import accuracy_score

trials = Trials()
best = fmin(fn=hyperparameter_tuning,
            space=space,
            algo=tpe.suggest,
            max_evals = 1,
            trials=trials)"""

'from sklearn.metrics import accuracy_score\n\ntrials = Trials()\nbest = fmin(fn=hyperparameter_tuning,\n            space=space,\n            algo=tpe.suggest,\n            max_evals = 1,\n            trials=trials)'

In [22]:
print(best)

NameError: name 'best' is not defined

In [ ]:
#new model with best parameters

In [23]:
model = CatBoostClassifier(loss_function='MultiClass',
                           objective='MultiClass',
                          eval_metric='Accuracy',
                          depth=6,
                          l2_leaf_reg=4,
                          iterations=150,
                          learning_rate=0.05,
                          colsample_bylevel=0.93,
                          min_child_samples=6)

In [24]:
import time

time_start = time.time()

model.fit(train_dataset)

time_end = time.time()
train_time = time_end - time_start
print("Train time:",train_time)

0:	learn: 0.5087237	total: 1.12s	remaining: 2m 47s
1:	learn: 0.5793581	total: 2.17s	remaining: 2m 40s
2:	learn: 0.5774278	total: 3.29s	remaining: 2m 41s
3:	learn: 0.5968184	total: 4.22s	remaining: 2m 34s
4:	learn: 0.5935670	total: 5.25s	remaining: 2m 32s
5:	learn: 0.5914490	total: 6.3s	remaining: 2m 31s
6:	learn: 0.6080915	total: 7.21s	remaining: 2m 27s
7:	learn: 0.6047564	total: 8.19s	remaining: 2m 25s
8:	learn: 0.6063559	total: 9.23s	remaining: 2m 24s
9:	learn: 0.6026878	total: 10.2s	remaining: 2m 22s
10:	learn: 0.5933443	total: 11.2s	remaining: 2m 21s
11:	learn: 0.6057389	total: 12.2s	remaining: 2m 20s
12:	learn: 0.6020112	total: 13.2s	remaining: 2m 18s
13:	learn: 0.6100988	total: 14.2s	remaining: 2m 17s
14:	learn: 0.6118457	total: 15.2s	remaining: 2m 16s
15:	learn: 0.6118839	total: 16.2s	remaining: 2m 15s
16:	learn: 0.6120251	total: 17.2s	remaining: 2m 14s
17:	learn: 0.6118294	total: 18.3s	remaining: 2m 14s
18:	learn: 0.6208664	total: 19.2s	remaining: 2m 12s
19:	learn: 0.6269366	to

In [25]:
import time
time_start = time.time()

pred = model.predict(X_test)

time_end = time.time()
test_time = time_end - time_start
print("test_time:",test_time)

test_time: 0.9364700317382812


In [26]:
from sklearn import metrics
from sklearn.metrics import classification_report

target_names = ['Analysis','Backdoor','DoS','Exploits','Fuzzers','Generic','Normal','Reconnaissance','Shellcode','Worms']
acc = metrics.accuracy_score(y_test,pred) * 100
f1 = metrics.f1_score(y_test, pred,average='weighted')* 100
pre = metrics.precision_score(y_test, pred, labels=None, pos_label=1, average='weighted') * 100 #DR
recall = metrics.recall_score(y_test, pred, labels=None, pos_label=1, average='weighted', sample_weight=None) * 100

print(classification_report(y_test,pred,target_names=target_names))
print("acc:",acc)
print("pre:",pre)
print("DR=recall:",recall)
print("f1:",f1)


                precision    recall  f1-score   support

      Analysis       0.06      0.76      0.11       803
      Backdoor       0.07      0.29      0.12       699
           DoS       0.00      0.04      0.01      4906
      Exploits       0.08      0.42      0.13     13358
       Fuzzers       0.37      0.67      0.48      7274
       Generic       1.00      0.97      0.99     64644
        Normal       1.00      0.80      0.89    665630
Reconnaissance       0.77      0.82      0.80      4196
     Shellcode       0.09      0.82      0.16       453
         Worms       0.05      0.87      0.10        52

      accuracy                           0.80    762015
     macro avg       0.35      0.65      0.38    762015
  weighted avg       0.97      0.80      0.87    762015

acc: 79.75551662368851
pre: 96.75901728440535
DR=recall: 79.75551662368851
f1: 86.93129339627018


In [27]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, pred)

TP = cm[0][0]
FP = cm[0][1]
FN = cm[1][0]
TN = cm[1][1]

FAR = FP / (FP + TN)
print(FAR)

0.4521276595744681
